In [42]:
import time

from randgen import gen_autocorrel
import scipy as sp

from edgedroid.models import *
import edgedroid.data as e_data
from gabriel_lego import LEGOTask, FrameResult

model = EdgeDroidModel(
    frame_trace=e_data.load_default_trace("square00", truncate=5),
    frame_model=ZeroWaitSamplingPolicy.from_default_data(),
    timing_model=MultiCurveFittingExecutionTimeModel(neuroticism=1.0)
)

task = LEGOTask(e_data.load_default_task("square00", truncate=5))

def emulate_run(model: EdgeDroidModel, task: LEGOTask, rtt_s_iter: Iterator[float]):
    for step_num, model_step in enumerate(model.play_steps()):
        print(f"step {step_num}")
        frame_timings: Optional[FrameTimings] = None
        prev_result: Optional[FrameResult] = None
        while True:
            try:
                model_frame = model_step.send(frame_timings)
            except StopIteration:
                if prev_result != FrameResult.SUCCESS:
                    raise Exception(prev_result)
                break
            
            ti = time.monotonic()
            prev_result = task.submit_frame(model_frame.frame_data)
            proc_time = time.monotonic() - ti
            
            rtt = next(rtt_s_iter)
            frame_timings = FrameTimings(rtt * 0.9, rtt * 0.1)
            
            

emulate_run(model, task, gen_autocorrel(dist=sp.stats.pareto, loc=0.6 * 5, rho=0.2, b=100.0))
model.model_step_metrics()

step 0
1.6003523764311107
step 1
1.3206848987189361
1.2689945968439096
1.255189363425718
1.2818303826193416
1.2685740414282796
1.2859477383845634
1.261868832526706
1.2530070758689633
1.2573151304516639
1.2693188621506224
1.2542825661157306
1.2512611411948615
1.2564706485858548
1.2578544212002323
1.2750560148445853
1.267632161241352
1.260042113629784
1.2528577141426434
1.2514677036975852
1.2571621652435174
1.2696287765940801
1.2672988964242595
1.2568301950986005
1.2552945430294433
1.252122035821236
1.2665305717704767
1.2603542423430782
1.2561468041310533
1.2811860778936315
1.2603794290738815
1.2596444675344238
1.262968835658794
1.255354849240579
1.257909236198282
1.2609094896190696
1.2601798100184232
1.264183274389123
1.2682580435763462
1.26454499524512
1.2783845042891182
1.2631316554718284
1.2722781192107036
1.2546344548886226
1.2511133238825156
1.2658956406289579
1.2557133757880141
1.251305640564073
1.2557094721621973
1.2835125454194243
1.2580508111490187
1.258743427014847
1.251774785

,step_start,step_start_monotonic,step_end,step_end_monotonic,first_frame_monotonic,last_frame_monotonic,last_frame_rtt,execution_time,step_duration,time_to_feedback,wait_time,frame_count
step_number,,,,,,,,,,,,
0,1.716785e+09,2.373773e+06,1.716785e+09,2.373773e+06,2.373773e+06,2.373773e+06,0.022245,0.000000,0.022337,0.022337,0.000092,1
1,1.716785e+09,2.373773e+06,1.716785e+09,2.373778e+06,2.373773e+06,2.373778e+06,0.284307,4.953603,5.260409,0.306806,0.022499,481
2,1.716785e+09,2.373778e+06,1.716785e+09,2.373783e+06,2.373778e+06,2.373783e+06,0.279398,5.064754,5.356833,0.292079,0.012681,63
3,1.716785e+09,2.373783e+06,1.716785e+09,2.373789e+06,2.373783e+06,2.373789e+06,0.312508,5.059687,5.471409,0.411722,0.099214,54
4,1.716785e+09,2.373789e+06,1.716785e+09,2.373794e+06,2.373789e+06,2.373794e+06,0.306674,4.776583,5.096074,0.319491,0.012817,74
5,1.716785e+09,2.373794e+06,1.716785e+09,2.373799e+06,2.373794e+06,2.373799e+06,0.285008,4.734288,5.273869,0.539581,0.254573,124
